In [1]:
import requests
from bs4 import BeautifulSoup as bs
import csv

imdb_url = "https://www.imdb.com/list/ls097840768/?sort=list_order,asc&st_dt=&mode=detail&page="
pages = [1,2,3]

game_class = "lister-item mode-detail"
image_class = "loadlate"
title_and_rank_class = "lister-item-header"
genre_class = "genre" 
ranking_class = "ipl-rating-star__rating"
votes_name = "mv"

def getImageGame(soup_game):
    image_tag = soup_game.find("img")

    if image_tag and image_class in image_tag.attrs:
        return image_tag[image_class]
    
    return None

def getTitleAndRankAndYear(soup_game):
    h3_tag = soup_game.find("h3", class_ = title_and_rank_class)

    if h3_tag != None:
        a_tag = h3_tag.find("a")
        span_tag = h3_tag.find("span")
        year_tag = h3_tag.find("span",class_="lister-item-year text-muted unbold")
        year = estrai_numeri(year_tag.text)
        title = a_tag.text
        rank = span_tag.text    
        return title, rank, year
    
    return None, None, None

def getGenreGame(soup_game):
    genre_tag = soup_game.find("span", class_ = genre_class)

    if genre_tag != None:
        genre_values = [genre.strip() for genre in genre_tag.text.split(',')]
        return genre_values
    return None

def getRankingGame(soup_game):
    tag = soup_game.find("span", class_ = ranking_class)
    if tag != None:
        return tag.text.strip()
    return None

def getDescriptionGame(soup_game):
    tag = soup_game.find("p", class_ = "")

    if tag != None:
        return tag.text.strip()
    return None

def getVotesGame(soup_game):
    span_tag = soup_game.find("span", attrs={"name": "nv"})
    if span_tag != None:
        return span_tag.text.strip()
    return None

def estrai_numeri(stringa):
 
    numeri = []

    for carattere in stringa:
        if carattere.isdigit():
            numeri.append(carattere)

    return "".join(numeri)

def dictGame(image, title, rank, genere, ranking, description, votes, year):
    game_info = {
        "Title": title,
        "Rank": rank,
        "Image": image,
        "Ranking": ranking,
        "Description": description,
        "Votes": votes,
        "Year": year
    }

    
    for i in range(3):
        try:
            if i < len(genere):
                game_info[f"Genre{i+1}"] = genere[i]
            else:
                game_info[f"Genre{i+1}"] = "-"
        except:
            game_info[f"Genre{i+1}"] = "-"

    return game_info

all_games_list = []

for page in pages:
    response = requests.get(imdb_url+str(page))
    if response.status_code == 200:
        soup = bs(response.content, "html.parser")
        
        games_list = soup.find_all("div", class_=game_class)

        for game in games_list:
            image = getImageGame(game)
            title, rank, year = getTitleAndRankAndYear(game)
            genere = getGenreGame(game)
            ranking = getRankingGame(game)
            description = getDescriptionGame(game)
            votes = getVotesGame(game)
            
            game_dict = dictGame(image, title, rank, genere, ranking, description, votes, year)
            all_games_list.append(game_dict)

    else:
        print("Failed to retrieve the page")



fields = ["Title", "Rank", "Image", "Ranking", "Description", "Votes", "Year", "Genre1", "Genre2",  "Genre3"]

filename = "test.csv"

with open(filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    for row in all_games_list:
        writer.writerow(row)
    